# Curso de aprendizaje automatizado
PCIC, UNAM

Machine Learning

Rodrigo S. Cortez Madrigal

<img src="https://pcic.posgrado.unam.mx/wp-content/uploads/Ciencia-e-Ingenieria-de-la-Computacion_color.png" alt="Logo PCIC" width="128" />   

### Tarea 2: Regresión y clasificación lineal

Un club del juego de Go recopiló los resultados de varias partidas entre diferentes jugadores, almacenados en el archivo partidas_entrenamiento.txt, con el objetivo de predecir el resultado de
partidas futuras, ejemplos de las cuales se encuentran en el archivo partidas_prueba.txt. Los archivos partidas_entrenamiento.txt y partidas_prueba.txt contienen 3 columnas: la primera
corresponde al identificador del jugador A, la segunda al identificador del jugador B y la tercera es
el resultado de la partida (1 si ganó el jugador A o 0 si ganó el jugador B). En el club hay un total
de D jugadores, por lo que cada identificador es un número entero entre 1 y D. La predicción del
resultado de un juego se puede plantear como un problema de clasificación: dados 2 jugadores (A y
B) se requiere predecir si A ganó (y = 1) o si fue B (y = 0).

In [2]:
import pandas as pd

from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import RepeatedKFold, cross_val_score

import plotly.express as px
import plotly.graph_objects as go

from tqdm import tqdm
import numpy as np

import sys
import os

### A

Entrena y evalúa un clasificador bayesiano ingenuo. Al ser un modelo generativo (modela
la probabilidad conjunta P (x, y)), es posible generar partidas artificiales con los parámetros
calculados. Genera nuevas partidas que sigan la distribución modelada

In [3]:
# Read juegos_entrenamiento.txt juegos_validacion.txt

df_train = pd.read_csv('regl_data/juegos_entrenamiento.txt', sep=' ')
df_train.columns = ['A', 'B', 'Win']
df_valid = pd.read_csv('regl_data/juegos_validacion.txt', sep=' ')
df_valid.columns = ['A', 'B', 'Win']

In [4]:
df_train.describe()

,A,B,Win
count,318.000000,318.000000,318.000000
mean,61.827044,64.226415,0.522013
std,42.756795,43.591656,0.500302
min,1.000000,1.000000,0.000000
25%,26.000000,23.000000,0.000000
50%,55.000000,55.500000,1.000000
75%,97.000000,105.000000,1.000000
max,142.000000,141.000000,1.000000


In [5]:
print(df_train.max())
print(df_train.min())
print(df_valid.max())
print(df_valid.min())

# Interesante...

A      142
B      141
Win      1
dtype: int64
A      1
B      1
Win    0
dtype: int64
A      132
B      136
Win      1
dtype: int64
A      4
B      4
Win    0
dtype: int64


In [6]:
# Plot A vs B
fig = px.scatter(df_train, x='A', y='B', color='Win')
fig.update_traces(marker=dict(size=5))
fig.update_layout(title='A vs B', xaxis_title='A', yaxis_title='B')
fig.show()

In [56]:
#Entrena y evalúa un clasificador bayesiano ingenuo. Al ser un modelo generativo (modela
#la probabilidad conjunta P (x, y)), es posible generar partidas artificiales con los parámetros
#calculados. Genera nuevas partidas que sigan la distribución modelada.

# Importar el modelo GaussianNaiveBayes de la Tarea 1
sys.path.append(os.path.abspath('../T1'))
from CustomNB import GaussianNaiveBayes

X_train = df_train[['A', 'B']].to_numpy()
y_train = df_train['Win'].to_numpy()
X_valid = df_valid[['A', 'B']].to_numpy()
y_valid = df_valid['Win'].to_numpy()

model = GaussianNaiveBayes(convariance=False)

model.fit(X_train, y_train)
y_pred = model.predict(X_valid)
accuracy = np.mean(y_pred == y_valid)

print(f'Accuracy: {accuracy:.2f}')

Accuracy: 0.68


Recordemos que a probabilidad conjunta es una medida en probabilidad que describe la probabilidad de que dos o más eventos ocurran simultáneamente. 

En NaiveBayes, la probabilidad conjunta se refiere a la probabilidad de observar un conjunto de características ( X ) y una clase ( y ), es decir, ( $P(X, y)$ ).

$$[ P(X, y) = P(y) \cdot P(X \mid y) ]$$

Donde:

( P(y) ): Es la probabilidad a priori de la clase ( y ).
( $P(X \mid y)$ ): Es la probabilidad condicional de las características ( X ) dado ( y ). En el caso de Gaussian Naive Bayes, se asume que cada característica sigue una distribución normal (gaussiana) y que las características son independientes entre sí.
El modelo calcula ( P(X, y) ) para cada clase ( y ) y luego utiliza la regla de Bayes para predecir la clase más probable ( y ) para un conjunto de características ( X ):

$$[ P(y \mid X) = \frac{P(X, y)}{P(X)} ]$$

Dado que ( P(X) ) es constante para todas las clases, el modelo simplemente maximiza ( P(X, y) ) para determinar la clase más probable.



In [ ]:
def generate_artificial_games(model, n_games=1000):
    """
    Generate artificial games using the trained model.
    """

    means = model.mean  # Array 2D: (n_classes, n_features)
    var = model.var  # Array 3D: (n_classes, n_features, n_features)
    class_priors = model.prior  # Probabilidades a priori de cada clase

    n_classes = len(model.classes)
    n_features = model.features
    samples = np.zeros((n_games, n_features))
    labels = np.zeros(n_games)

    for i in range(n_games):
        class_idx = np.random.choice(n_classes, p=class_priors)
        #random.normal(loc=0.0, scale=1.0, size=None)#
        sample = np.random.normal(loc=means[class_idx], scale=var[class_idx])
        sample = np.clip(sample, X_train.min(), X_train.max())
        #sample = np.maximum(np.random.multivariate_normal(means[class_idx], covariances[class_idx]), 0)
        samples[i] = sample
        labels[i] = class_idx

    samples_df = pd.DataFrame(samples, columns=['A', 'B']).astype(int)
    labels_df = pd.DataFrame(labels, columns=['Win'])
    samples_df = pd.concat([samples_df, labels_df], axis=1)
    samples_df.columns = ['A', 'B', 'Win']
    samples_df['Win'] = samples_df['Win'].astype(int)

    return samples_df

artificial_games = generate_artificial_games(model, n_games=1000)

fig = px.scatter(artificial_games, x='A', y='B', color='Win',
                 title='Artificial Games', labels={'A': 'A', 'B': 'B', 'Win': 'Win'})
fig.update_traces(marker=dict(size=5))
fig.update_layout(title='Artificial Games',
                  xaxis_title='A',
                  yaxis_title='B')
fig.show()

In [60]:
model = GaussianNaiveBayes(convariance=True)

model.fit(X_train, y_train)
y_pred = model.predict(X_valid)
accuracy = np.mean(y_pred == y_valid)

print(f'Accuracy: {accuracy:.2f}')

def generate_artificial_games(model, n_games=1000):
    """
    Generate artificial games using the trained model.
    """

    means = model.mean  # Array 2D: (n_classes, n_features)
    covariances = model.covariances  # Array 3D: (n_classes, n_features, n_features)
    class_priors = model.prior  # Probabilidades a priori de cada clase

    n_classes = len(model.classes)
    n_features = model.features
    samples = np.zeros((n_games, n_features))
    labels = np.zeros(n_games)

    for i in range(n_games):
        class_idx = np.random.choice(n_classes, p=class_priors)
        sample = np.random.multivariate_normal(means[class_idx], covariances[class_idx])
        sample = np.clip(sample, X_train.min(), X_train.max())
        #sample = np.maximum(np.random.multivariate_normal(means[class_idx], covariances[class_idx]), 0)
        samples[i] = sample
        labels[i] = class_idx

    samples_df = pd.DataFrame(samples, columns=['A', 'B']).astype(int)
    labels_df = pd.DataFrame(labels, columns=['Win'])
    samples_df = pd.concat([samples_df, labels_df], axis=1)
    samples_df.columns = ['A', 'B', 'Win']
    samples_df['Win'] = samples_df['Win'].astype(int)

    return samples_df

# Generate 1000 artificial games
artificial_games = generate_artificial_games(model, n_games=1000)

artificial_games = generate_artificial_games(model, n_games=1000)

fig = px.scatter(artificial_games, x='A', y='B', color='Win',
                 title='Artificial Games', labels={'A': 'A', 'B': 'B', 'Win': 'Win'})
fig.update_traces(marker=dict(size=5))
fig.update_layout(title='Artificial Games',
                  xaxis_title='A',
                  yaxis_title='B')
fig.show()

Accuracy: 0.68


## B

Entrena y evalúa un clasificador de regresión logística6. Debido a que ambos atributos son
categóricos, es necesario cambiar la codificación. Explica el procedimiento y la lógica de la
codificación que realizaste. Visualiza los valores de los parámetros del modelo de regresión
logística y discute qué interpretación tendrían de acuerdo a la codificación realizada. Grafica
las curvas ROC y de precisión-exhaustividad y reporta sus áreas bajo la curva.